#  3.5新增

## @overload

In [ ]:
from typing import Literal, overload


@overload
def transform(data: str, mode: Literal["split"]) -> list[str]: ...


@overload
def transform(data: str, mode: Literal["upper"]) -> str: ...


def transform(data: str, mode: Literal["split", "upper"]) -> list[str] | str: # 这里管道符"|"是 3.10 新特性
    if mode == "split":
        return data.split()
    else:
        return data.upper()


split_words = transform("hello world", "split")  
split_words[0]  # pyright静态检查通过 （pyright 是一个静态类型检查工具，通过 uv add pyright 或 pip install pyright 安装）

upper_words = transform("hello world", "upper")  
upper_words.lower()  # pyright静态检查通过 

upper_words.append("!") # pyright静态检查失败
                        # error: Cannot access attribute "append" for class "str"
                        # Attribute "append" is unknown (reportAttributeAccessIssue)

# 3.6新增

##  变量注解

In [ ]:
# 在 3.6 中使用变量注解 [citation:3][citation:15][citation:19]
name: str = "Alice"
age: int = 30
is_active: bool = True

# 可以在定义变量时直接添加注解，也可以先注解后赋值 [citation:3][citation:15]
salary: float
salary = 5000.50

# 类型检查器可以根据注解发现潜在的类型错误
# age = "thirty" # 类型检查器会标记错误

# 3.7 新增

## 未来注释

In [ ]:
def build_user() -> User: # 报错：未定义“User”
    ...

def build_user() -> "User": # 解决使用时还未定义
    ...

class User:
    ...

In [ ]:
from __future__ import annotations # 解决使用时还未定义

def build_user() -> User:
    ...

class User:
    ...

# 3.8新增

## 协议

In [ ]:
from typing import Protocol


class Quackable(Protocol):
    def quack(self,) -> None: 
        ...


class Duck:
    def quack(self):
        print("Quack!")


class Dog:
    def bark(self):
        print("Woof!")


def run_quack(obj: Quackable):
    obj.quack()


run_quack(Duck())  # Works!
run_quack(Dog())  # Fails during TYPE CHECKING (not runtime)

# 3.9新增

## 缓存

In [ ]:
from functools import cache

@cache
def fib(n):
    return n if n < 2 else fib(n-1) + fib(n-2)

## 字典操作

In [ ]:
dict1 = {"a": 1, "b": 2}
dict2 = {"c": 3, "d": 4}
dict3 = {"a": 5, "e": 6}

# 合并 (创建新字典)，右边的字典会覆盖左边同名的键 [citation:2][citation:14][citation:20]
merged_dict_39 = dict1 | dict2 | dict3
print(merged_dict_39) # 输出: {'a': 5, 'b': 2, 'c': 3, 'd': 4, 'e': 6}

# 更新 (修改原字典)，使用 |= 运算符 [citation:2][citation:14][citation:20]
dict1 |= dict2
print(dict1) # 输出: {'a': 1, 'b': 2, 'c': 3, 'd': 4}

# 3.10新增

## 结构模式匹配（match）

In [ ]:
def get_day_type(day):
    match day:
        case ("Monday"
            | "Tuesday"
            | "Wednesday"
            | "Thursday"
            | "Friday"):
            return "Weekday"
        case "Saturday" | "Sunday":
            return "Weekend"

def get_point_desc(point):        
    match point:
        case (0, 0):
            return "Origin"
        case (0, y):
            return f"Y-axis at {y}"
        case (x, 0):
            return f"X-axis at {x}"
        case (x, y):
            return f"Point at ({x}, {y})"

def get_tempreature_desc(temperature):        
    match temperature:
        case temp if temp < 0:
            return "Freezing"
        case temp if temp < 20:
            return "Cold"
        case temp if temp < 30:
            return "Warm"
        case _:
            return "Hot"

# 3.11新增

## “Self”类型

In [ ]:
from typing import Self


class User:

    def build(self) -> Self:  
        return self

# 3.12新增

## 泛型

In [ ]:
class KVStore[K: str | int, V]:
    def __init__(self) -> None:
        self.store: dict[K, V] = {}

    def get(self, key: K) -> V:
        return self.store[key]

    def set(self, key: K, value: V) -> None:
        self.store[key] = value

kv = KVStore[str, int]()
kv.set("one", 1)
kv.set("two", 2)
kv.set("three", 3)

## @override

In [ ]:
from typing import override

class Base:
    
    def greet(self) -> None:
        print("Hello")

class Derived(Base):
    
    @override # 标记此方法意在覆盖 Base.greet
    def greet(self) -> None:
        print("Hi")

class AnotherDerived(Base):
    
    # 如果这里写成了 gret 而没有使用 @override，类型检查器不会报错
    def gret(self) -> None:
        print("Hey")

class YetAnotherDerived(Base):
    
    @override
    def gret(self) -> None:
        # 如果这里使用 @override 但超类中没有 gret 方法，
        # 类型检查器（如 Pyright/Mypy）会在这里报告错误 [citation:5][citation:12]
        print("Oops")

# 其他 

##  字符串字面量（限定输入）

In [ ]:
def set_color(color: Literal["red", "blue", "green"]) -> None:
    ...

set_color("red")
set_color("blue")
set_color("green")
set_color("fuchsia")  # Argument of type "Literal['fuchsia']" cannot be assigned to parameter "color"

##  仅关键字和仅位置参数

In [ ]:
def foo(a, *, b):  # `*`后面的参数必须作为关键字参数传递。
	...
	
foo(a=1, b=2) 
foo(1, b=2) 
foo(1, 2) # Cannot use positional for keyword-only parameter

def bar(a, /, b): # `/` 之前的参数必须按位置传递，不能用作关键字参数。
	...
	
bar(1, 2) 
bar(1, b=2) 
bar(a=1, b=2) # Cannot use keyword for positional-only parameter

##  Slot内存优化

In [ ]:
class Optimized:  
	__slots__ = ['x', 'y']  
	def __init__(self, x, y):  
		self.x = x  
		self.y = y

obj = Optimized(10, 20)  
# obj.z = 30 # This would raise an error because 'z' is not in __slots__

## 迭代器

In [ ]:
class MyIterator:  
	def __init__(self, max):  
		self.max = max  
		self.current = 0  
	  
	def __iter__(self):  
		return self  
	  
	def __next__(self):  
		if self.current < self.max:  
			self.current += 1  
			return self.current  
		else:  
			raise StopIteration  
  
iterator = MyIterator(5)  
for value in iterator:  
	print(value)


# 当然如下方式也可以，就是不OOP
def countdown(n):
    while n > 0:
        yield n
        n -= 1
for i in countdown(5):
    print(i)

## 描述符

In [ ]:
class Descriptor: 
	def __get__(self, instance, owner): 
		return instance._value 
		
	def __set__(self, instance, value): 
		instance._value = value * 2 
		
class MyClass: 
	attribute = Descriptor() 
	
obj = MyClass() 
obj.attribute = 10 
print(obj.attribute) # Output: 20

## 元类、抽象基类、数据类

In [ ]:
# 元类 - 99%的日常开发用例中，都不需要使用它
class Meta(type):
    def __new__(cls, name, bases, namespace):
        print(f"Creating class {name}")
        new_namespace = {}
        for key, val in namespace.items():
            if isinstance(val, int):
                val *= 2 # 这里对类的属性进行2倍放大修改
            new_namespace[key] = val
        return super().__new__(cls, name, bases, new_namespace)


class MyClass(metaclass=Meta):
    x = 5
    y = 10

print(MyClass.x)  # 5被元类放大2倍，输出为10
print(MyClass.y)  # 同理，y输出20



# 抽象基类
from abc import ABC, abstractmethod


class Animal(ABC):
    @abstractmethod
    def sound(self):
        pass


class Dog(Animal):
    def sound(self):
        return "Woof!"


dog = Dog()
print(dog.sound())




# 数据类
from dataclasses import dataclass


@dataclass
class Point:
    x: int
    y: int


p = Point(10, 20)
print(p)  # Output: Point(x=10, y=20)

## for-else

In [ ]:
class server:
    def check_availability(self):
        return True


servers = [server() for _ in range(10)]
backup_server = server()


def deploy_application(primary_server): ...


for server in servers:
    if server.check_availability():
        primary_server = server
        break
else:
    # Use the backup server if no server was found
    primary_server = backup_server

# Continue execution with whatever server we found
deploy_application(primary_server)

## 海象运算符

In [ ]:
def get_user_input():
    return input('Enter something: ')

if response := get_user_input():
    print('You pressed:', response)
else:
    print('You pressed nothing')

## or运算

In [ ]:
import os

def get_user_info():
    username = os.getenv("USERNAME")
    full_name = os.getenv("FULL_NAME")
    first_name = os.getenv("FIRST_NAME")
    return username, full_name, first_name

username, full_name, first_name = get_user_info()
display_name = username or full_name or first_name or "Anonymous"

## f-string

In [2]:
import os
from datetime import datetime

time = datetime.now()
run_id = 7890
progress = 0.75
total_samples = 123456789
accuracy = 0.925732954
loss = 0.05346592
memory = 10000000000

print(f"{' [ Run Status ] ':=^50}")
print(f"[{time:%H:%M:%S}] Training Run {run_id=} status: {progress:.1%}")
print(f"Summary: {total_samples:,} samples processed")
print(f"Accuracy: {accuracy:.4f} | Loss: {loss:#.3g}")
print(f"Memory: {memory / 1e9:+.2f} GB")

================= [ Run Status ] =================
[11:53:13] Training Run run_id=7890 status: 75.0%
Summary: 123,456,789 samples processed
Accuracy: 0.9257 | Loss: 0.0535
Memory: +10.00 GB


In [3]:
name = "Monor"
age = 18

print(f"{name=}, {age=}")

name='Monor', age=18


In [4]:
pi = 3.141592653589793
avogadro = 6.02214076e23
big_num = 12345678901234567890
num = 42

print(f"Pi: {pi:.2f}")
print(f"Avogadro: {avogadro:.2e}")
print(f"Big Number: {big_num:,}")
print(f"Hex: {num:#0x}")
print(f"Number: {num:09}")

Pi: 3.14
Avogadro: 6.02e+23
Big Number: 12,345,678,901,234,567,890
Hex: 0x2a
Number: 000000042


In [5]:
word = "Python"

print(f"Left: |{word:<10}|")
print(f"Right: |{word:>10}|")
print(f"Center: |{word:^10}|")
print(f"Center *: |{word:*^10}|")

Left: |Python    |
Right: |    Python|
Center: |  Python  |
Center *: |**Python**|


In [6]:
now = datetime.now()

print(f"Date: {now:%Y-%m-%d}")
print(f"Time: {now:%H:%M:%S}")

Date: 2025-04-24
Time: 11:55:20
